In [ ]:
# All needed imports
import pickle
import cv2
import glob
import os
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#### 1) Camera Calibration
The camera calibration is done with the help of serveral images of the same chessboard using the same camera taken
from different angels. The next two methods extract the image and object points from the given imagaes and calculate the cameramatrix and distoration matrix for the given camera.

In [ ]:
# 1. Camera Calibration
def extract_image_and_object_points(image_dir, chessboard_x=9, chessboard_y=6):
    """
    Extracts the image points and onbject points from the given image_dir

    Args:
        image_dir:    The path glob which can be used to find the images
        chessboard_x: The chessboard corners in the x axis
        chessboard_y: The chessboard corners in the y axis

    Returns:
        The pickle file that conatin both of these data.
    """
    objp = np.zeros((chessboard_x * chessboard_y, 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_x, 0:chessboard_y].T.reshape(-1, 2)
    objpoints = []
    imgpoints = []
    images = glob.glob(image_dir)
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(
            gray, (chessboard_x, chessboard_y), None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    return imgpoints, objpoints


def get_image_and_distortion_matrix(image_dir,
                                    test_image,
                                    chessboard_x=9,
                                    chessboard_y=6):
    """
    Returns the image matrix and ditortion matrix for the given image, image points and object points

    Args:
        image_dir:    The path glob which can be used to find the images
        test_image:   The array like image or PIL image
        chessboard_x: The chessboard corners in the x axis
        chessboard_y: The chessboard corners in the y axis

    Returns:
        The pickel file that contains the distorations
    """
    imgpoints, objpoints = extract_image_and_object_points(image_dir)
    img_size = (test_image.shape[1], test_image.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints,
                                                       img_size, None, None)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    dist_pickle['imgpoints'] = imgpoints
    dist_pickle['objpoints'] = objpoints
    pickle.dump(dist_pickle, open("calibrate_dist_pickle.p", "wb"))

In [ ]:
img = cv2.imread("camera_cal/calibration14.jpg")
get_image_and_distortion_matrix('camera_cal/*.jpg', img)

#### 2) Undistort
Having caclulated the camera and distortion matrix, every image created by the same camrea can be distorted and transformed. This is done by the next command.

In [ ]:
# 2. Apply the calibration to remove distoration on the given raw images
def undistort_and_transform(img,
                            chessboard_x=9,
                            chessboard_y=6,
                            calibrate_pickle='calibrate_dist_pickle.p'):
    """
    Undistort the given image with the help of calibrate pickel created before.

    Args:
        img: The array like image or PIL image
        calibrate_pickle: The path to the pickle file that should contain the image matrix and distorations

    Returns:
        Returns the undistorted image.
    """
    dist_pickle = pickle.load(open(calibrate_pickle, "rb"))
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    grayscale = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(
        undist, (chessboard_x, chessboard_y), None)
    M = None
    warped = np.copy(undist)
    if ret == True:
        cv2.drawChessboardCorners(undist, (chessboard_x, chessboard_y),
                                  corners, ret)
        image_size = undist.shape[1::-1]
        src_points = np.float32([
            corners[0], corners[chessboard_x - 1], corners[-1],
            corners[-chessboard_x]
        ])
        offset = 50
        dst_points = np.float32(
            [[offset, offset], [image_size[0] - offset, offset],
             [image_size[0] - offset, image_size[1] - offset],
             [offset, image_size[1] - offset]])
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        warped = cv2.warpPerspective(
            undist, M, image_size, flags=cv2.INTER_LINEAR)
    return warped, M

def undistort_image(img, calibrate_pickle='calibrate_dist_pickle.p'):
    """
    Undistorts the given image with the calibration pickel
    
    Args:
        img: The image that should be used is the array like image or PIL
        calibrate_pickle: The pickle file that should be used for the source of calibration
        
    Returns:
        The undistorted image
    """
    dist_pickle = pickle.load(open(calibrate_pickle, "rb"))
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [ ]:
img = cv2.imread("camera_cal/calibration1.jpg")
top_down, perspective_M = undistort_and_transform(img)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down)
ax2.set_title('Undistorted', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = cv2.imread(path)
    image_name = os.path.basename(path)
    top_down = undistort_image(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir + "undistorted_" + image_name, top_down)
    top_down = cv2.cvtColor(top_down, cv2.COLOR_BGR2RGB)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original-' + image_name, fontsize=25)
    ax2.imshow(top_down)
    ax2.set_title('Undistorted - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 3) Create Threshold binary image
To create a threshold binary image the following steps should be taken:

- Undisort the image
- Use the HLS color space (Mainly the S) to limit the colors in the image
- Use Sobel gradient in direction and magnitude
- Combine the gradient and limited color image togehther and with this create the binary image

Results of this operation can be found in the `output_image/binary_*`

In [ ]:
def hls_select_s(img, thresh=(0, 255)):
    """
    Applies the HLS threshold on the given image.
    
    Args:
        img: The image that should be used
        thresh: The threshold for the given image
        
    Returns:
        img
    """
    img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = img_hls[:, :, 2]
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary


def sobel_process(img, orient='x', thresh_min=20, thresh_max=100):
    """
    Adds the sobel process to the given image.
    
    Args:
        img: The image like array
        oreint: The Sobel orientation, could be x or y
        thresh_min: The minimum threshold for the sobel
        thresh_max: The maximum threshold for the sobel
        
    Return:
        image like array that is processed.
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = cv2.normalize(abs_sobel, None, 0.0, 255.0, cv2.NORM_MINMAX,
                                 cv2.CV_64F)
    sxbinary = cv2.inRange(scaled_sobel, thresh_min, thresh_max)
    retval, sxbinary = cv2.threshold(sxbinary, 250, 1.0, cv2.THRESH_BINARY)
    return sxbinary


def direction_threshold(img, sobel_kernel=3, thresh=(0, np.pi / 2)):
    """
    Creates the direction threshold with the help of Sobel
    
    Args:
        img: The image that should be manipulated
        sobel_kernel: The size of the sobel kernel
        thresh: The threshold value for the direction threshold
    
    Returns:
        img
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    sobel_dir = np.arctan2(abs_sobely, abs_sobelx)
    sbinary = cv2.inRange(sobel_dir, thresh[0], thresh[1])
    retval, sbinary = cv2.threshold(sbinary, thresh[1] - 0.1, 1.0,
                                    cv2.THRESH_BINARY)
    return sbinary


def magnitude_threshold(img, sobel_kernel=3, mag_threshold=(0, 255)):
    """
    Creates the magnitude threshold with the help of Sobel
    
    Args:
        img: The image that should be manipulated
        sobel_kernel: The size of the sobel kernel
        mag_threshold: The threshold magniuted
    
    Returns:
        img
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    mag_sobel = np.sqrt(abs_sobelx**2 + abs_sobely**2)
    scaled_sobel = cv2.normalize(mag_sobel, None, 0.0, 255.0, cv2.NORM_MINMAX,
                                 cv2.CV_64F)
    sbinary = cv2.inRange(scaled_sobel, mag_threshold[0], mag_threshold[1])
    retval, sbinary = cv2.threshold(sbinary, 250, 1.0, cv2.THRESH_BINARY)
    return sbinary


def combined_threshold(image,
                       sobel_kernel=15,
                       dir_thresh=(0.0, np.pi / 2),
                       mag_threshold=(0, 255)):
    """
    Creates the combined threshold for the replacement of the canny detection algorithm
    
    Args:
        image: The array like image that should be used.
        sobel_kernel: The size of the sobel kernel
        dir_thresh: The threshold that should be used for the sobel function
        mag_threshold: The theshold for the magnitude and normal sobel process
        
    Returns:
        image
    """
    gradx = sobel_process(image, 'x', mag_threshold[0], mag_threshold[1])
    grady = sobel_process(image, 'y', mag_threshold[0], mag_threshold[1])
    mag_binary = magnitude_threshold(image, sobel_kernel, mag_threshold)
    dir_binary = direction_threshold(image, sobel_kernel, dir_thresh)
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) &
                                              (dir_binary == 1))] = 1
    return combined


def get_binary_image(image,
                     mag_thresh=(0, 255),
                     hls_thresh=(0, 255)):
    """
    Creates the binary image from the given image with help of Sobel and HLS colour space
    
    Args:
    
        image: The image like array or the PIL image
        mag_thresh: The threshold that should be used for the soble magnitude
        hls_thresh: The threshold that should be used for the hls colour limiting
    
    Returns:
        The filtered binary image
    """
    hsl_image = hls_select_s(image, hls_thresh)
    threshold_image = sobel_process(image, 'x', mag_thresh[0], mag_thresh[1])
    combined_binary = np.zeros_like(threshold_image)
    combined_binary[(hsl_image == 1) | (threshold_image == 1)] = 1
    return combined_binary

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    img = undistort_image(img)
    top_down = get_binary_image(img, (20,100),(170,255))
    cv2.imwrite(output_dir + "binary_" + image_name, top_down * 255)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original - ' + image_name, fontsize=25)
    ax2.imshow(top_down, cmap='Greys_r')
    ax2.set_title('Binary - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 4) Perspective transform
Using a transform matrix, the binary image will be perspective transformed to a bird-eye view.

In [ ]:
def transform_bird_eye(img, offset = 50):
    """
    Transform the ROI (Region of Interest) of the given image to bird eye
    
    Args:
        img: The array like image or PIL image
        offset: The offset for the transformation
        
    Return:
            The transformed image
            The transform matrix
            The inverse of transform matrix
    """
    imshape = img.shape
    img_size = (imshape[1], imshape[0])
    left_point =  (1.5 * imshape[1]/8, imshape[0]- 50)
    source_apex1 = (imshape[1]/2 - 50 , imshape[0]/2 + 85 )
    source_apex2 = (imshape[1]/2 + 50, imshape[0]/2 + 85 )
    right_point = (6.5 * imshape[1]/8, imshape[0]- 50)
    source = np.float32([[left_point, source_apex1, source_apex2, right_point]])
    dest_left_point =  (1.5 * imshape[1]/8  + offset, imshape[0])
    dest_apex1 = (1.5 * imshape[1]/8 + offset, 0)
    dest_apex2 = (6.5 * imshape[1]/8 - offset, 0)
    dest_right_point = (6.5 * imshape[1]/8 - offset, imshape[0])
    dest = np.float32([[dest_left_point, dest_apex1, dest_apex2, dest_right_point]])
    M = cv2.getPerspectiveTransform(source, dest)
    Minv = cv2.getPerspectiveTransform(dest, source)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, M, Minv

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    img = undistort_image(img)
    top_down, M, Minv = transform_bird_eye(img)
    top_down_print = cv2.cvtColor(top_down, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir + "bird_eye_" + image_name, top_down_print)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original - ' + image_name, fontsize=25)
    ax2.imshow(top_down)
    ax2.set_title('Bird Eye - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    img = undistort_image(img)
    top_down = get_binary_image(img, (20,100),(170,255))
    top_down, M, Minv= transform_bird_eye(top_down)
    cv2.imwrite(output_dir + "bird_eye_binary_" + image_name, top_down * 255)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original - ' + image_name, fontsize=25)
    ax2.imshow(top_down,  cmap='Greys_r')
    ax2.set_title('Bird Eye - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 5) Find the lane pixels
After preproccesing the image, it will be fitted to the lane pixel finder, it uses the following steps:

- Create a historgram of the images, bottom part so the left and right expected area of the line are found
- Use this and the sliding window approach to find the whole line.
- Draw it.

In [ ]:
def find_lane_pixels(binary_warped):
    """
    Finds the lane pixels in a binary warped image
    
    Args:
        binary_warped: The binary warped array_like image that should be used to find line
        
    Returns:
        The arrays containing points for the left and right image
    """
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0] // 2:, :], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] // 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0] // nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low),
                      (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low),
                      (win_xright_high, win_y_high), (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) &
                          (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) &
                           (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped, draw_poly = True):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    try:
        left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty**2 + right_fit[
            1] * ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1 * ploty**2 + 1 * ploty
        right_fitx = 1 * ploty**2 + 1 * ploty
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    if draw_poly is True:
        left_points = np.array(list(zip(left_fitx, ploty)),  np.int32)
        right_points = np.array(list(zip(right_fitx, ploty)),  np.int32)
        cv2.polylines(out_img, [left_points], False, (255,255,0),2)
        cv2.polylines(out_img, [right_points], False, (255,255,0), 2)
    return left_fit, right_fit, left_fitx, right_fitx, ploty, out_img

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    img = undistort_image(img)
    top_down = get_binary_image(img, (20, 150), (150, 255))
    top_down, M, Minv = transform_bird_eye(top_down)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(top_down)
    to_print = cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir + "lane_found_" + image_name, to_print)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original - ' + image_name, fontsize=25)
    ax2.imshow(out_img)
    ax2.set_title('Pixel Lane - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

#### 6) Determine the curvature of the lane and vehicle position with respect to center.
The curvature of the line is calculate with the given formula.

In [ ]:
def generate_data(leftx, rightx, ploty, ym_per_pix, xm_per_pix):
    '''
    Generates fake data to use for calculating lane curvature.
    In your own project, you'll ignore this function and instead
    feed in the output of your lane detection algorithm to
    the lane curvature calculation.
    '''
    dest = abs(leftx[-1] - rightx[-1])*xm_per_pix
    center_of_image = abs(xm_per_pix * 660)
    dest_from_center = abs(dest - center_of_image)
    left_fit_cr  = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix , 2)
    right_fit_cr = np.polyfit(ploty* ym_per_pix, rightx * xm_per_pix, 2)
    return ploty, left_fit_cr, right_fit_cr, dest_from_center

def measure_curvature_real(left_fit, right_fit, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    
    Args:
        left_fit: The line that is fitted to the left lane
        right_fit: The line that is fitted tot the right lane
        ploty: The ploty image.
        
    Returns:
        left_curverad, right_curverad
    '''
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    ploty, left_fit_cr, right_fit_cr, dest_from_center = generate_data(left_fit, right_fit, ploty, ym_per_pix, xm_per_pix)
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2 * left_fit_cr[0] * y_eval * ym_per_pix + left_fit_cr[1])**2)** (3/2))/ np.absolute((2 * left_fit_cr[0]))
    right_curverad = ((1 + (2 * right_fit_cr[0] * y_eval * ym_per_pix  + right_fit_cr[1])**2)** (3/2))/ np.absolute((2 * right_fit_cr[0]))
    return left_curverad, right_curverad, dest_from_center

def sanitize_curvature(left_curverad, right_curverad, max_curvature = 20000):
    """
    Sanitizes the Curvature for the given fit.
    
    Args:
        left_curverad: The left lane curvature in m
        right_curverad: The right lane curvature in m
        max_curvature: The maximum curvature
        
    Returns:
        The value of the curvature in m
    """
    curvature = (left_curverad + right_curverad)/2
    if curvature > max_curvature:
        return max_curvature
    if curvature > 1000:
        return (curvature / 100) * 100
    else:
        return (curvature / 50) * 50

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    img = undistort_image(img)
    top_down = get_binary_image(img, (20, 150), (150, 255))
    top_down, M, Minv= transform_bird_eye(top_down)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(top_down)
    left_curverad, right_curverad, dest_from_center =  measure_curvature_real(left_fitx, right_fitx, ploty)
    print("Curvature: ", image_name, left_curverad, right_curverad)
    print("Dest:", dest_from_center)

#### 7) Warp the detected lane boundaries back onto the original image.
With the help of the transformation matrix used before, the detected lanelines are added to the original image.

In [ ]:
def draw_lane_boundires(img, sobel_thresh = (20, 150), hls_thresh=(150, 255)):
    """
    Draw lane boundries on the image
    
    Args:
        img: The img array
        sobel_thresh: The sobel threshold
        hls_thresh: The threshold for the hls image filter
        
    Return:
        img
    """
    undist = undistort_image(img)
    top_down = get_binary_image(undist, sobel_thresh, hls_thresh)
    warped, M, Minv = transform_bird_eye(top_down)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(warped)
    left_curverad, right_curverad, dest_from_center =  measure_curvature_real(left_fitx, right_fitx, ploty)
    curvature = sanitize_curvature(left_curverad, right_curverad)
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    curvature_text = "Curvature:% 6.0fm" % curvature
    position_text = "Position: % 5.2fm" % dest_from_center
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, curvature_text,(100,70), font, 2,(0,0,0),2)
    cv2.putText(result, position_text, (150,120), font, 2,(0,0,0),2)
    return result

In [ ]:
test_image_dir = "test_images/*.jpg"
output_dir = "output_images/"
images = glob.glob(test_image_dir)
for path in images:
    img = mpimg.imread(path)
    image_name = os.path.basename(path)
    top_down = draw_lane_boundires(img, (20, 150), (150, 255))
    cv2.imwrite(output_dir + "lane_drawn_" + image_name, top_down)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original - ' + image_name, fontsize=25)
    ax2.imshow(top_down)
    ax2.set_title('Pixel Lane - ' + image_name, fontsize=25)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Load our image - this should be a new frame since last time!
binary_warped = mpimg.imread('warped-example.jpg')

# Polynomial fit values from the previous frame
# Make sure to grab the actual values from the previous step in your project!
left_fit = np.array([ 2.13935315e-04, -3.77507980e-01,  4.76902175e+02])
right_fit = np.array([4.17622148e-04, -4.93848953e-01,  1.11806170e+03])

def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty

def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_poly, right_poly, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

# Run image through the pipeline
# Note that in your project, you'll also want to feed in the previous fits
result = search_around_poly(binary_warped)

# View your output
plt.imshow(result)

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line(object):
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line for the left lane
        self.recent_xfitted_left = deque(maxlen = 10)
        # x values of the last n fits of the line for the right lane
        self.recent_xfitted_right = deque(maxlen =10)
        # average x values of the fitted line over the last n iterations for the left lane
        self.best_x_left = None
        # average x values of the fitted line over the last n iteration for the right lane
        self.best_x_right = None
        # polynomial coefficients averaged over the last n iterations for the left lane
        self.best_fit_left = None 
        # polynomial coefficients averaged over the last n iterations for the right lane
        self.best_fit_right = None
        # polynomial coefficients for the last n fits for the left lane
        self.recent_fits_left = deque(maxlen =10)
        # polynomial coefficients for the last n fits for the right lane
        self.recent_fits_right = deque(maxlen =10)
        # polynomial coefficients for the most recent fit for the left lane
        self.current_fit_left = [np.array([False])] 
        # polynomial coeficients for the most recent fir for the right lane
        self.current_fit_right = [np.array([False])]
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits for the left lane
        self.diffs_left = np.array([0,0,0], dtype='float')
        #difference in fit coefficients between last and new fits for the right lane
        self.diffs_right = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels for left lane
        self.allx_left = None 
        #x values for detected line pixels for left right
        self.allx_right = None
        #y values for detected line pixels
        self.ally = None
        
    def get_default_frame_data(self):
        """
        Returns the default frame_data for the given time ine the line
        
        
        Returns:
            frame_data: The FrameData object that should be used.
        """
        frame_data = FrameData()
        frame_data.left_fit = self.best_fit_left
        frame_data.right_fit = self.best_fit_right
        frame_data.left_fitx = self.best_x_left
        frame_data.right_fitx = self.best_x_right
        frame_data.ploty = ploty
        frame_data.curvature = self.radius_of_curvature
        frame_data.dest_from_center = self.line_base_pos
        return frame_data

class FrameData(object):
    """
    The data that is extracted form the given frame.
    """
    def __init__(self):
        self.left_fit = None
        self.left_fitx = None 
        self.right_fit = None 
        self.right_fitx = None
        self.ploty = None 
        self.curvature = None
        self.right_curverad = None
        self.left_curverad = None
        self.dest_from_center = None

In [ ]:
MAX_ACCEPTABLE_WIDTH = 1.0 # in meters
MAX_ACCEPTABLE_PARALEL = 5.0 # in percent
MAX_ACCEPTABLE_LINE_MOVED = .2 # in meters

def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty


def brute_force_fit(img, sobel_thresh = (20, 150), hls_thresh=(150, 255)):
    """
    This is the brute force fit that is used only when the normal look a head fitting does not work.
    
    Args:
        img:          The array like image or PIL image
        sobel_thresh: The size of the sobel threshold
        hls_thresh:   The threshold for the hls binary
        
    Returns:
        frame_data: The frame data which contains all the frame related data.
        M:          The Matrix used to the transform the image to bird_eye view
        Minv:       The inverse of the matrix that is used to transform images to bird_eye view
    """
    img = undistort_image(img)
    top_down = get_binary_image(img, sobel_thresh, hls_thresh)
    top_down, M, Minv = transform_bird_eye(top_down)
    left_fit, right_fit, left_fitx, right_fitx, ploty, out_img = fit_polynomial(top_down)
    left_curverad, right_curverad, dest_from_center =  measure_curvature_real(left_fitx, right_fitx, ploty)
    frame_data = FrameData()
    frame_data.left_fit = left_fit
    frame_data.right_fit = right_fit
    frame_data.left_fitx = left_fitx
    frame_data.right_fitx = right_fitx
    frame_data.ploty = ploty
    frame_data.right_curverad = right_curverad
    frame_data.left_curverad = left_curverad
    frame_data.curvature = sanitize_curvature(left_curverad, right_curverad)
    frame_data.dest_from_center = dest_from_center
    return frame_data, M, Minv

def simple_fit(img, line,  sobel_thresh= (20, 150), hls_thresh=(150, 255)):
    """
    This is is the simple fit which does not used the sliding window approach to find the lane lines
    
    Args:
        img:  The array like image or PIL image
        line: The line object which contains the history of the operation
        
    Returns:
        frame_data: The frame data that can be used for the sanity_check or being added to the list of lines
    """
    left_fit = line.current_fit_left
    right_fit = line.current_fit_right
    binary_image = get_binary_image(img, sobel_thresh, hls_thresh)
    binary_warped, M, Minv = transform_bird_eye(binary_image)
    margin = 100
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    left_fit_poly, right_fit_poly, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    left_curverad, right_curverad, dest_from_center =  measure_curvature_real(left_fitx, right_fitx, ploty)
    frame_data = FrameData()
    frame_data.left_fit = left_fit_poly
    frame_data.right_fit = right_fit_poly
    frame_data.left_fitx = left_fitx
    frame_data.right_fitx = right_fitx
    frame_data.ploty = ploty
    frame_data.right_curverad = right_curverad
    frame_data.left_curverad = left_curverad
    frame_data.curvature = sanitize_curvature(left_curverad, right_curverad)
    frame_data.dest_from_center = dest_from_center
    return frame_data

def sanitiy_check(frame_data, line):
    """
    Checks if the given line is a valid line with the sanity check.
    
    Args:
        image: The image that should be used.
        line: The line object that should be used for the line
        
    Returns:
        bool
    """
    xm_per_pix = 3.7/700
    lane_width = abs(frame_data.left_fitx[-1] - frame_data.right_fitx[-1]) * xm_per_pix
    if np.abs(lane_width - 3.7) > MAX_ACCEPTABLE_WIDTH:
        return False
    curve_differs = np.abs((frame_data.left_curverad - frame_data.right_curverad) / frame_data.left_curverad)
    if curve_differs > MAX_ACCEPTABLE_PARALEL:
        return False
    lane_moved = np.abs(frame_data.dest_from_center - line.line_base_pos)
    if lane_moved > MAX_ACCEPTABLE_LINE_MOVED:
        return False
    return True

def add_line(line, frame_data):
    """
    Adds a new line to the given buffer.
    
    Args:
        line: The empty line object which will be field with the data of the
        frame_data: The data that is extracted from the given frame
        
    Returns:
        line: The mainpulate line buffer
    """
    if len(line.recent_xfitted_left) == 0:
        ## If the buffer is empty fill it with the first value.
        line.detected = True
        line.recent_xfitted_left.append(frame_data.left_fitx)
        line.recent_xfitted_right.append(frame_data.right_fitx)
        line.best_x_left = frame_data.left_fitx
        line.best_x_right = frame_data.right_fitx
        line.best_fit_left = frame_data.left_fit
        line.best_fit_right = frame_data.right_fit
        line.recent_fits_left.append(frame_data.left_fit)
        line.recent_fits_right.append(frame_data.right_fit)
        line.current_fit_left = frame_data.left_fit
        line.current_fit_right = frame_data.right_fit
        line.radius_of_curvature = frame_data.curvature
        line.line_base_pos = frame_data.dest_from_center
        line.allx_left = frame_data.left_fitx
        line.allx_right = frame_data.right_fitx
        line.ally = frame_data.ploty
    else:
        line.detected = True
        line.recent_xfitted_left.append(frame_data.left_fitx)
        line.recent_xfitted_right.append(frame_data.right_fitx)
        line.best_x_left = np.mean(np.array(line.recent_xfitted_left),axis=0)
        line.best_x_right = np.mean(np.array(line.recent_xfitted_right),axis=0)
        line.best_fit_left = np.mean(np.array(line.recent_fits_left),axis=0)
        line.best_fit_right = np.mean(np.array(line.recent_fits_right),axis=0)
        line.recent_fits_left.append(frame_data.left_fit)
        line.recent_fits_right.append(frame_data.right_fit)
        line.diffs_left = np.absolute(line.current_fit_left - frame_data.left_fit)
        line.diffs_right = np.absolute(line.current_fit_right - frame_data.right_fit)
        line.current_fit_left = frame_data.left_fit
        line.current_fit_right =frame_data.right_fit
        line.radius_of_curvature = frame_data.curvature
        line.line_base_pos = frame_data.dest_from_center
        line.allx_left = frame_data.left_fitx
        line.allx_right = frame_data.right_fitx
        line.ally = frame_data.ploty
    return line

def draw_frame(img, frame_data):
    """
    Draws the frame_data retrieved data on the given image.
    
    Args:
        img:        The array like image or image file
        frame_data: The frame_data calculated data
    
    Return:
        img
    """
    undist = undistort_image(img)
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    top_down, M, Minv = transform_bird_eye(gray)
    warp_zero = np.zeros_like(top_down).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    left_fitx = frame_data.left_fitx
    right_fitx = frame_data.right_fitx
    ploty = frame_data.ploty
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    curvature_text = "Curvature:% 6.0fm" % frame_data.curvature
    position_text = "Position: % 5.2fm" % frame_data.dest_from_center
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, curvature_text,(100,70), font, 2,(0,0,0),2)
    cv2.putText(result, position_text, (150,120), font, 2,(0,0,0),2)
    return result

def pipeline(image, **params):
    """
    The pipeline for the image processing
    
    Args:
        image: The array like image or PIL image
        params: The parameters that should be used.
    
    Returns: The finished image.
    """
    line = params["line"]
    if len(line.recent_xfitted_left) == 0:
        frame_data, M, Minv = brute_force_fit(image)
        add_line(line, frame_data)
    else:
        frame_data = simple_fit(image, line)
        if sanitiy_check(frame_data, line):
            add_line(line, frame_data)
    frame_data = line.get_default_frame_data()
    return draw_frame(image, frame_data)


In [ ]:
# The buffer the results of the line calculation in the last 10 iteration
line = Line()
params = {"line":line}

def process_image(image):
    """
    With this it is possble to pass the parameters to the pipeline.
    
    Args:
        image: The image that should be used, array like image or PIL image
    
    Return:
        The manipulated image
    """
    return pipeline(image, ** params)


In [ ]:
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML
output_clip = 'project_view_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
advanced_lane = clip1.fl_image(process_image)
%time advanced_lane.write_videofile(output_clip, audio=False)